In [46]:
import optuna
import plotly.io as pio

# Load the study
study = optuna.load_study(study_name="nn_control_alloc", storage="sqlite:///../optuna_studies/nn_control_alloc.db")

#RENAME - TEMPORARIO
# Define the mapping of old to new (shorter) names
rename_map = {
    "optimizer_learning_rate": "opt_LR",
    'leakyReLU_negative_slope': 'LReLU_nslope'
}

renamed_trials = []
for trial in study.trials:
    if trial.state != optuna.trial.TrialState.COMPLETE:
        continue

    # Rename params
    new_params = {rename_map.get(k, k): v for k, v in trial.params.items()}

    # Rename distributions to match renamed params
    new_distributions = {
        rename_map.get(k, k): v for k, v in trial.distributions.items()
    }

    # Create the new trial
    new_trial = optuna.trial.create_trial(
        params=new_params,
        distributions=new_distributions,
        value=trial.value,
        intermediate_values=trial.intermediate_values,
        user_attrs=trial.user_attrs,
        system_attrs=trial.system_attrs,
        state=trial.state
    )
    renamed_trials.append(new_trial)

# Create a new study and add renamed trials
renamed_study = optuna.create_study(direction=study.direction)
renamed_study.add_trials(renamed_trials)
study = renamed_study


[I 2025-05-07 20:54:10,014] A new study created in memory with name: no-name-5c3047a6-36c2-41a6-b89c-9c75bc24a56e


In [47]:
# Visualizations
fig1 = optuna.visualization.plot_optimization_history(study)
fig2 = optuna.visualization.plot_parallel_coordinate(study)
fig3 = optuna.visualization.plot_param_importances(study)
fig4 = optuna.visualization.plot_slice(study)
fig5 = optuna.visualization.plot_contour(study)

In [48]:
# Display interactively
fig1.show()

In [49]:
fig2.show()

In [50]:
fig3.show()

In [51]:
fig4.show()

In [52]:

# Improve layout: increase size, spacing, font
fig5.update_layout(
    width=1200,
    height=1200,
    font=dict(size=14),
    margin=dict(l=60, r=60, t=60, b=60),
)

# Optional: Rotate axis labels if overlapping
fig5.update_xaxes(tickangle=45)

# Rotate x-axis and y-axis labels for clarity
#for axis in fig5.layout:
#    if isinstance(fig5.layout[axis], dict) and "tickangle" in fig5.layout[axis]:
#        fig5.layout[axis]["tickangle"] = 45

fig5.update_xaxes(tickangle=45)
fig5.update_yaxes(tickangle=0)

# Show the cleaned-up plot
fig5.show()

In [53]:
print(f"Number of trials: {len(study.trials)}")

completed = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
print(f"Completed trials: {len(completed)}")

Number of trials: 32
Completed trials: 32


In [54]:
filtered_trials = [t for t in study.trials if t.params.get("n_layers") == 2]
sub_study = optuna.create_study()
sub_study.add_trials(filtered_trials)
fig = optuna.visualization.plot_parallel_coordinate(sub_study)
fig.show()

[I 2025-05-07 20:54:12,555] A new study created in memory with name: no-name-7eba29d1-7a12-407d-ab53-f209c683b63d


In [55]:
fig.update_traces(line=dict(colorscale="Viridis"))  # Other options: "Inferno", "Cividis", "Bluered", etc.
fig.show()